In [ ]:
!pip install matplotlib
!pip install torch
!pip install colpali-engine
!pip install torchvision
!pip install wandb

In [22]:
import wandb
import os# from torchvision import transforms
from data_collection.datasets import IAMDataset
from PIL import Image
from torchvision import transforms
import torch
from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor
import random
import wandb
import os

In [8]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [2]:
lines_txt_path = 'data/IAM/ascii/lines.txt'
base_image_path = 'data/IAM/lines'

if not os.path.exists(lines_txt_path) or not os.path.exists(base_image_path):
    print(f"Error: Dataset files not found.")
    print(f"Please ensure '{lines_txt_path}' and '{base_image_path}' exist.")
    exit()


try:
    dataset = IAMDataset(
        lines_txt_path=lines_txt_path,
        base_image_path=base_image_path,
    )
    print(f"Dataset loaded successfully with {len(dataset)} samples.")
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

Dataset loaded successfully with 13353 samples.


In [54]:
def create_datasets(
    lines_txt_path='data/IAM/ascii/lines.txt',
    base_image_path='data/IAM/lines',
    transform_fns=None
):
    if transform_fns is None:
        transform_fns = {'default': None}

    datasets = {}
    for name, fn in transform_fns.items():
        datasets[name] = IAMDataset(
            lines_txt_path=lines_txt_path,
            base_image_path=base_image_path,
            transform=fn
        )
    return datasets

In [58]:
def evaluate_datasets(
    datasets,
    models,
    processor,
    device,
    scores,
    num_samples
):
    """
    Evaluate each model on each dataset and log metrics to W&B.

    Args:
        datasets: Dict of IAMDataset instances.
        models: Dict mapping model names to models.
        processor: The ColPaLi processor for image/text -> embeddings.
        device: Torch device (e.g., torch.device('cuda')).
        scores: Dict mapping score names to scoring functions.
        num_samples: Number of samples to evaluate per dataset.
    """
    for ds_name, ds in datasets.items():
        for model_name, model in models.items():
            run_name = f"{model_name}_{ds_name}"
            wandb.init(project="colpali_evaluation", name=run_name)
            model.to(device).eval()

            for i in range(min(num_samples, len(ds))):
                image, text = ds[i]
                try:
                    # Process inputs
                    processed_image = processor.process_images([image]).to(device)
                    processed_text = processor.process_queries([text]).to(device)

                    # Inference
                    with torch.no_grad():
                        img_emb = model(**processed_image)
                        txt_emb = model(**processed_text)

                    # Apply each scoring function
                    log_data = {
                        "dataset": ds_name,
                        "model": model_name,
                        "sample_index": i,
                    }
                    for score_name, score_fn in scores.items():
                        score_value = score_fn(txt_emb, img_emb).item()
                        log_data[score_name] = score_value

                    wandb.log(log_data)

                except Exception as e:
                    print(f"Error at ds={ds_name}, model={model_name}, idx={i}: {e}")

            wandb.finish()

In [41]:
model_name = "nomic-ai/colnomic-embed-multimodal-7b"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ColQwen2_5.from_pretrained(model_name).to(device).eval()
processor = ColQwen2_5_Processor.from_pretrained(model_name)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
scores = {
    "similarity_score": processor.score_multi_vector
}
transfns = {"default":None}
datasets = create_datasets(transform_fns=transfns)

evaluate_datasets(
    datasets=datasets,
    models={"base":model},
    processor=processor,
    device=device,
    scores = scores,
    num_samples=100
)

sample_index,▁▁▁▁▁▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
similarity_score,▃▃▂▃▃▃▆▁▅▃▃▄▂▃▂▄▅▅▆▃▇▄▄▅▁▄▁▄▄▄█▅▇▁▄▃▅▃▅▃
dataset,default
model,base
sample_index,99
similarity_score,19.76352
